<a href="https://colab.research.google.com/github/haddybhaiya/sem-i-con/blob/main/conv_mobilenetv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install timm albumentations torchmetrics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 23.9 MB/s eta 0:00:00


# old one

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import Compose
import timm
import os
import numpy as np

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

DATASET_PATH = "/content/drive/MyDrive/synthetic_dataset"
TEACHER_PATH = "/content/convnext_sem.pth"
STUDENT_SAVE_PATH = "/content/drive/MyDrive/mobilenetv3_distilled.pth"

CLASSES = ["clean","bridge","cmp","crack","open","ler","via","other"]
NUM_CLASSES = len(CLASSES)

IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 40
WARM_UP_EPOCHS = 5

ALPHA = 0.3        # hard vs soft loss
TEMPERATURE = 6.0


In [ ]:
class GrayImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        path, label = self.samples[index]
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img.astype(np.float32) / 255.0
        img = torch.tensor(img).unsqueeze(0)  # (1,H,W)
        return img, label


In [ ]:
import cv2

dataset = GrayImageFolder(DATASET_PATH)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)


In [ ]:
teacher = timm.create_model(
    "convnext_tiny",
    pretrained=False,
    in_chans=1,
    num_classes=NUM_CLASSES
)

teacher.load_state_dict(torch.load(TEACHER_PATH, map_location=DEVICE))
teacher.to(DEVICE)
teacher.eval()

for p in teacher.parameters():
    p.requires_grad = False


In [ ]:
student = timm.create_model(
    "mobilenetv3_small_100",
    pretrained=True,
    in_chans=1,
    num_classes=NUM_CLASSES
)

student.to(DEVICE)
student.train()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/10.2M [00:00<?, ?B/s]

MobileNetV3(
  (conv_stem): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): Hardswish()
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
        (bn1): BatchNormAct2d(
          16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
        (aa): Identity()
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): ReLU(inplace=True)
          (conv_expand): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (gate): Hardsigmoid()
        )
        (conv_pw): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): B

In [ ]:
class DistillationLoss(nn.Module):
    def __init__(self, alpha, temperature):
        super().__init__()
        self.alpha = alpha
        self.T = temperature
        self.ce = nn.CrossEntropyLoss()
        self.kl = nn.KLDivLoss(reduction="batchmean")

    def forward(self, student_logits, teacher_logits, labels):
        hard_loss = self.ce(student_logits, labels)

        soft_student = F.log_softmax(student_logits / self.T, dim=1)
        soft_teacher = F.softmax(teacher_logits / self.T, dim=1)

        soft_loss = self.kl(soft_student, soft_teacher) * (self.T ** 2)

        return self.alpha * hard_loss + (1 - self.alpha) * soft_loss


In [ ]:
optimizer = torch.optim.AdamW(student.parameters(), lr=5e-5, weight_decay = 1e-4)
criterion = DistillationLoss(ALPHA, TEMPERATURE)


In [ ]:
for epoch in range(EPOCHS):
    total_loss = 0.0

    for imgs, labels in loader:
        imgs = imgs.to(DEVICE)
        labels = labels.to(DEVICE)

        with torch.no_grad():
            teacher_logits = teacher(imgs)

        student_logits = student(imgs)
        loss = criterion(student_logits, teacher_logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{EPOCHS}] Loss: {total_loss/len(loader):.4f}")


Epoch [1/25] Loss: 4.9216
Epoch [2/25] Loss: 3.2270
Epoch [3/25] Loss: 3.1104
Epoch [4/25] Loss: 2.9715
Epoch [5/25] Loss: 2.9064
Epoch [6/25] Loss: 2.8694
Epoch [7/25] Loss: 2.8593
Epoch [8/25] Loss: 2.8226
Epoch [9/25] Loss: 2.8021
Epoch [10/25] Loss: 2.7774
Epoch [11/25] Loss: 2.7703
Epoch [12/25] Loss: 2.7706
Epoch [13/25] Loss: 2.7751
Epoch [14/25] Loss: 2.7527
Epoch [15/25] Loss: 2.7529
Epoch [16/25] Loss: 2.7476
Epoch [17/25] Loss: 2.7431
Epoch [18/25] Loss: 2.7387
Epoch [19/25] Loss: 2.7353
Epoch [20/25] Loss: 2.7328
Epoch [21/25] Loss: 2.7302
Epoch [22/25] Loss: 2.7274
Epoch [23/25] Loss: 2.7304
Epoch [24/25] Loss: 2.7254
Epoch [25/25] Loss: 2.7219


In [ ]:
torch.save(student.state_dict(), STUDENT_SAVE_PATH)
print("✅ Distilled MobileNetV3 saved:", STUDENT_SAVE_PATH)


✅ Distilled MobileNetV3 saved: /content/drive/MyDrive/mobilenetv3_distilled.pth


In [ ]:
student.eval()
correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in loader:
        imgs = imgs.to(DEVICE)
        labels = labels.to(DEVICE)
        out = student(imgs)
        preds = out.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print("Training-set accuracy:", correct / total)


Training-set accuracy: 0.25


# new try

In [ ]:
!pip install -q timm torchvision scikit-learn


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import timm
import os
import numpy as np


In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

DATASET_ROOT = "/content/drive/MyDrive/synthetic_dataset"  # change if needed
NUM_CLASSES = 8
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 40
WARMUP_EPOCHS = 5

CLASS_NAMES = [
    "clean","bridge","cmp","crack",
    "open","ler","via","other"
]

print("Using device:", DEVICE)


Using device: cuda


In [ ]:
train_tfms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.ToTensor(),
])

val_tfms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])


In [ ]:
full_dataset = torchvision.datasets.ImageFolder(
    root=DATASET_ROOT,
    transform=train_tfms
)

num_total = len(full_dataset)
num_val = int(0.15 * num_total)
num_train = num_total - num_val

train_dataset, val_dataset = torch.utils.data.random_split(
    full_dataset, [num_train, num_val]
)

val_dataset.dataset.transform = val_tfms

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print("Train:", len(train_dataset), " Val:", len(val_dataset))


Train: 2040  Val: 360


In [ ]:
teacher = timm.create_model(
    "convnext_tiny",
    pretrained=True,
    in_chans=1,
    num_classes=NUM_CLASSES
)

teacher.load_state_dict(
    torch.load("/content/convnext_sem.pth", map_location=DEVICE)
)

teacher.to(DEVICE)
teacher.eval()

for p in teacher.parameters():
    p.requires_grad = False

print("Teacher loaded & frozen")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

Teacher loaded & frozen


In [ ]:
student = timm.create_model(
    "mobilenetv3_small_100",
    pretrained=True,
    in_chans=1,
    num_classes=NUM_CLASSES
)

student.to(DEVICE)
print("Student model ready")


model.safetensors:   0%|          | 0.00/10.2M [00:00<?, ?B/s]

Student model ready


In [ ]:
class DistillationLoss(nn.Module):
    def __init__(self, alpha=0.3, temperature=6.0):
        super().__init__()
        self.alpha = alpha
        self.temperature = temperature
        self.ce = nn.CrossEntropyLoss(label_smoothing=0.05)
        self.kl = nn.KLDivLoss(reduction="batchmean")

    def forward(self, s_logits, t_logits, labels):
        loss_ce = self.ce(s_logits, labels)

        T = self.temperature
        loss_kl = self.kl(
            F.log_softmax(s_logits / T, dim=1),
            F.softmax(t_logits / T, dim=1)
        ) * (T * T)

        return self.alpha * loss_ce + (1 - self.alpha) * loss_kl


In [ ]:
def freeze_student_backbone(model):
    for name, p in model.named_parameters():
        if "classifier" not in name:
            p.requires_grad = False

def unfreeze_student(model):
    for p in model.parameters():
        p.requires_grad = True


In [ ]:
criterion = DistillationLoss(alpha=0.3, temperature=6.0)

optimizer = torch.optim.AdamW(
    student.parameters(),
    lr=5e-5,
    weight_decay=1e-4
)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=EPOCHS
)


In [ ]:
freeze_student_backbone(student)

for epoch in range(EPOCHS):
    student.train()
    total_loss = 0
    correct = 0
    total = 0

    if epoch == WARMUP_EPOCHS:
        print("🔥 Unfreezing student backbone")
        unfreeze_student(student)

    for imgs, labels in train_loader:
        imgs = imgs.to(DEVICE)
        labels = labels.to(DEVICE)

        optimizer.zero_grad()

        with torch.no_grad():
            t_logits = teacher(imgs)

        s_logits = student(imgs)
        loss = criterion(s_logits, t_logits, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = s_logits.argmax(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    scheduler.step()

    train_acc = correct / total
    avg_loss = total_loss / len(train_loader)

    # ---------- Validation ----------
    student.eval()
    v_correct = 0
    v_total = 0

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs = imgs.to(DEVICE)
            labels = labels.to(DEVICE)

            out = student(imgs)
            preds = out.argmax(1)

            v_correct += (preds == labels).sum().item()
            v_total += labels.size(0)

    val_acc = v_correct / v_total

    print(
        f"Epoch [{epoch+1}/{EPOCHS}] "
        f"Loss {avg_loss:.3f} "
        f"Train {train_acc:.3f} "
        f"Val {val_acc:.3f}"
    )


Epoch [1/40] Loss 25.473 Train 0.170 Val 0.114
Epoch [2/40] Loss 24.467 Train 0.173 Val 0.133
Epoch [3/40] Loss 23.218 Train 0.175 Val 0.133
Epoch [4/40] Loss 22.354 Train 0.177 Val 0.144
Epoch [5/40] Loss 21.184 Train 0.181 Val 0.161
🔥 Unfreezing student backbone
Epoch [6/40] Loss 7.241 Train 0.259 Val 0.228
Epoch [7/40] Loss 4.214 Train 0.256 Val 0.228
Epoch [8/40] Loss 3.773 Train 0.254 Val 0.228
Epoch [9/40] Loss 3.498 Train 0.254 Val 0.228
Epoch [10/40] Loss 3.340 Train 0.254 Val 0.228
Epoch [11/40] Loss 3.260 Train 0.254 Val 0.228
Epoch [12/40] Loss 3.224 Train 0.254 Val 0.228
Epoch [13/40] Loss 3.151 Train 0.254 Val 0.228
Epoch [14/40] Loss 3.097 Train 0.254 Val 0.228
Epoch [15/40] Loss 3.075 Train 0.254 Val 0.228
Epoch [16/40] Loss 3.044 Train 0.254 Val 0.228
Epoch [17/40] Loss 3.024 Train 0.254 Val 0.228
Epoch [18/40] Loss 3.000 Train 0.254 Val 0.228
Epoch [19/40] Loss 2.991 Train 0.254 Val 0.228
Epoch [20/40] Loss 2.968 Train 0.254 Val 0.228
Epoch [21/40] Loss 2.954 Train 0.2

In [ ]:
torch.save(student.state_dict(), "mobilenetv3_distilled.pth")
print("Saved distilled student model")


Saved distilled student model


In [ ]:
student.eval()
correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs = imgs.to(DEVICE)
        labels = labels.to(DEVICE)
        out = student(imgs)
        preds = out.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print("Training-set accuracy:", correct / total)

Training-set accuracy: 0.22777777777777777


# try 3

In [ ]:
# ===============================
# 📦 INSTALLS
# ===============================
!pip install -q timm torchvision tqdm

# ===============================
# 📚 IMPORTS
# ===============================
import os, random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import timm
from tqdm import tqdm

# ===============================
# ⚙️ CONFIG
# ===============================
DATASET_ROOT = "/content/drive/MyDrive/synthetic_dataset"   # 8 class folders inside
NUM_CLASSES = 8
BATCH_SIZE = 32
IMG_SIZE = 224
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ===============================
# 🖼️ TRANSFORMS (GRAYSCALE ONLY)
# ===============================
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

# ===============================
# 📂 LOAD DATASET (AUTO SPLIT)
# ===============================
full_ds = datasets.ImageFolder(DATASET_ROOT, transform=transform)

val_size = int(0.2 * len(full_ds))
train_size = len(full_ds) - val_size

train_ds, val_ds = random_split(full_ds, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

print("Classes:", full_ds.classes)

# ===============================
# 🧠 MODELS
# ===============================

# ---- Teacher (ConvNeXt) ----
teacher = timm.create_model(
    "convnext_tiny",
    pretrained=False,
    in_chans=1,
    num_classes=NUM_CLASSES
)

teacher.load_state_dict(torch.load("/content/convnext_sem.pth", map_location=DEVICE))
teacher.to(DEVICE)
teacher.eval()

# ---- Student (MobileNetV3) ----
student = timm.create_model(
    "mobilenetv3_small_100",
    pretrained=True,
    in_chans=1,
    num_classes=NUM_CLASSES
)
student.to(DEVICE)

# ===============================
# 📏 EVAL FUNCTION
# ===============================
def evaluate(model):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            pred = model(x).argmax(1)
            correct += (pred == y).sum().item()
            total += y.size(0)
    return correct / total

# ===============================
# 🔥 PHASE 1 — WARMUP TRAINING
# ===============================
optimizer = torch.optim.AdamW(student.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()

print("\n🔥 PHASE 1 — WARMUP\n")

for epoch in range(30):
    student.train()
    for x, y in train_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        loss = criterion(student(x), y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    val_acc = evaluate(student)
    print(f"[Warmup {epoch+1}/30] Val {val_acc:.3f}")

# ===============================
# 🧠 PHASE 2 — SAFE DISTILLATION
# ===============================

# Freeze backbone
for name, p in student.named_parameters():
    if "classifier" not in name and "head" not in name:
        p.requires_grad = False

optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, student.parameters()),
    lr=1e-4
)

T = 4.0
LAMBDA = 0.05

print("\n🧠 PHASE 2 — SAFE DISTILLATION\n")

for epoch in range(6):
    student.train()
    for x, y in train_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)

        with torch.no_grad():
            t_logits = teacher(x)

        s_logits = student(x)

        loss_ce = criterion(s_logits, y)
        loss_kd = F.kl_div(
            F.log_softmax(s_logits / T, dim=1),
            F.softmax(t_logits / T, dim=1),
            reduction="batchmean"
        ) * (T * T)

        loss = loss_ce + LAMBDA * loss_kd

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    val_acc = evaluate(student)
    print(f"[KD {epoch+1}/6] Val {val_acc:.3f}")

# ===============================
# 💾 SAVE FINAL STUDENT
# ===============================
torch.save(student.state_dict(), "mobilenetv3_sem_distilled.pth")
print("\n✅ Saved mobilenetv3_sem_distilled.pth")



Classes: ['bridge', 'clean', 'cmp', 'crack', 'ler', 'open', 'other', 'via']

🔥 PHASE 1 — WARMUP

[Warmup 1/30] Val 0.981
[Warmup 2/30] Val 0.981
[Warmup 3/30] Val 0.883
[Warmup 4/30] Val 0.969
[Warmup 5/30] Val 0.975
[Warmup 6/30] Val 0.971
[Warmup 7/30] Val 0.975
[Warmup 8/30] Val 0.996
[Warmup 9/30] Val 1.000
[Warmup 10/30] Val 0.996
[Warmup 11/30] Val 1.000
[Warmup 12/30] Val 1.000
[Warmup 13/30] Val 1.000
[Warmup 14/30] Val 1.000
[Warmup 15/30] Val 1.000
[Warmup 16/30] Val 1.000
[Warmup 17/30] Val 1.000
[Warmup 18/30] Val 1.000
[Warmup 19/30] Val 1.000
[Warmup 20/30] Val 1.000
[Warmup 21/30] Val 1.000
[Warmup 22/30] Val 1.000
[Warmup 23/30] Val 1.000
[Warmup 24/30] Val 1.000
[Warmup 25/30] Val 1.000
[Warmup 26/30] Val 1.000
[Warmup 27/30] Val 1.000
[Warmup 28/30] Val 1.000
[Warmup 29/30] Val 1.000
[Warmup 30/30] Val 1.000

🧠 PHASE 2 — SAFE DISTILLATION

[KD 1/6] Val 0.910
[KD 2/6] Val 0.990
[KD 3/6] Val 0.994
[KD 4/6] Val 0.985
[KD 5/6] Val 0.996
[KD 6/6] Val 0.998

✅ Saved mobilen